In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("data/year_prediction.csv")
df.head()

,label,TimbreAvg1,TimbreAvg2,TimbreAvg3,TimbreAvg4,TimbreAvg5,TimbreAvg6,TimbreAvg7,TimbreAvg8,TimbreAvg9,...,TimbreCovariance69,TimbreCovariance70,TimbreCovariance71,TimbreCovariance72,TimbreCovariance73,TimbreCovariance74,TimbreCovariance75,TimbreCovariance76,TimbreCovariance77,TimbreCovariance78
0,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903


In [2]:
df.dtypes

label                   int64
TimbreAvg1            float64
TimbreAvg2            float64
TimbreAvg3            float64
TimbreAvg4            float64
                       ...   
TimbreCovariance74    float64
TimbreCovariance75    float64
TimbreCovariance76    float64
TimbreCovariance77    float64
TimbreCovariance78    float64
Length: 91, dtype: object

In [3]:
#As recommended by the owners of the dataset
train_df=df.iloc[:463715]
test_df=df.iloc[463715:]

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
from tqdm import tqdm

class YearNetwork(nn.Module):
    def __init__(self,input_size):
        super().__init__()

        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.BatchNorm1d(input_size),
            nn.Linear(input_size,128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Linear(32,16),
            nn.ReLU(),
            nn.BatchNorm1d(16),
            nn.Linear(16,1)
        )

    def forward(self,x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)

        return x

class YearDataset(torch.utils.data.Dataset):
    def __init__(self,data):
        self.features=data.drop(['label'],axis=1)
        self.labels=data[['label']]
        #print(f"features: {self.features.shape}\nlabels: {self.labels.shape}\n")

    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,idx):

        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        feature_items=self.features.iloc[[idx]].to_numpy()
        label_items=self.labels.iloc[[idx]].to_numpy()

        x=torch.tensor(feature_items,dtype=torch.float32)
        y=torch.tensor(label_items,dtype=torch.float32)       

        return x,y

def train_loop(model,dataloader,optimizer,epoch):
    model.train()
    for batch in tqdm(dataloader, desc=f"Epoch {epoch}: "):
        inputs, labels = batch
        labels=torch.flatten(labels)
        pred = model(inputs)
        
        loss=F.mse_loss(torch.ravel(pred), labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

def test_loop(model,dataloader):
    model.eval()
    size = len(dataloader)
    test_loss=0
    with torch.no_grad():
        for inputs,labels in tqdm(dataloader, desc="Testing: "):
            pred=model(inputs)

            loss_item=F.mse_loss(torch.ravel(pred),torch.ravel(labels)).item()
            test_loss+=loss_item
            
            
    test_loss=test_loss / size
    print(f'Test loss: {test_loss}')


In [18]:
from torch.utils.data import DataLoader
import torch.optim as optim

lr=.005
batch_size=32
epochs=10

model=YearNetwork(df.shape[1]-1)
train_dataset,test_dataset=YearDataset(train_df),YearDataset(test_df)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True,drop_last=True)

optimizer=optim.Adam(model.parameters(),lr=lr)

In [6]:
batch = next(iter(train_dataloader))
features,labels=batch
print(f"Feature batch shape: {features.size()}")
print(f"Labels batch shape: {labels.size()}")
feat = features[:2]
label = labels[:2]
pred=model(feat)
print(torch.ravel(pred))
print(torch.flatten(label))

Feature batch shape: torch.Size([32, 1, 90])
Labels batch shape: torch.Size([32, 1, 1])
tensor([0.1151, 0.2845], grad_fn=<ViewBackward0>)
tensor([2006., 2010.])


In [30]:
def train_network(model,train_dataloader,test_dataloader,optimizer):
    for epoch in range(epochs):
        print("==========================================")
        train_loop(model=model,dataloader=train_dataloader,optimizer=optimizer,epoch=epoch+1)
        test_loop(model=model,dataloader=test_dataloader)

In [7]:
train_network(model=model,train_dataloader=train_dataloader,test_dataloader=test_dataloader,optimizer=optimizer)

Testing: 100%|██████████| 1613/1613 [00:15<00:00, 101.75it/s]


Test loss: 81.63167117828918, size: 1613


Testing: 100%|██████████| 1613/1613 [00:15<00:00, 104.37it/s]


Test loss: 80.01809287736376, size: 1613


Testing: 100%|██████████| 1613/1613 [00:15<00:00, 101.89it/s]


...

Test loss: 78.42303012840806, size: 1613


Testing: 100%|██████████| 1613/1613 [00:16<00:00, 99.37it/s] 


Test loss: 83.35907750091269, size: 1613


Testing: 100%|██████████| 1613/1613 [00:16<00:00, 98.97it/s] 

Test loss: 78.60077188684566, size: 1613


In [31]:
batch_size=256
epochs=5
torch.manual_seed(42)
model256=YearNetwork(df.shape[1]-1)
train_dataloader256 = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,drop_last=True)
test_dataloader256 = DataLoader(test_dataset, batch_size=batch_size, shuffle=True,drop_last=True)

optimizer=optim.Adam(model256.parameters(),lr=lr)

In [32]:
#larger batch size
train_network(model=model256,train_dataloader=train_dataloader256,test_dataloader=test_dataloader256,optimizer=optimizer)

Testing: 100%|██████████| 201/201 [00:14<00:00, 13.78it/s]


Test loss: 43349.65782416045, size: 201


Testing: 100%|██████████| 201/201 [00:14<00:00, 13.64it/s]


Test loss: 77.61157816796754, size: 201


Testing: 100%|██████████| 201/201 [00:15<00:00, 13.02it/s]


Test loss: 77.30072729385907, size: 201


Testing: 100%|██████████| 201/201 [00:13<00:00, 14.73it/s]


Test loss: 77.58990698667309, size: 201


Testing: 100%|██████████| 201/201 [00:14<00:00, 13.83it/s]

Test loss: 76.74773851081507, size: 201


In [10]:
batch_size=512
lr=.002
epochs=15
model512=YearNetwork(df.shape[1]-1)
train_dataloader512 = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,drop_last=True)
test_dataloader512 = DataLoader(test_dataset, batch_size=batch_size, shuffle=True,drop_last=True)

optimizer=optim.Adam(model512.parameters(),lr=lr)

In [11]:
#Even larger batch size
train_network(model=model512,train_dataloader=train_dataloader512,test_dataloader=test_dataloader512,optimizer=optimizer)

Testing: 100%|██████████| 100/100 [00:13<00:00,  7.20it/s]


Test loss: 3481823.7825, size: 100


Testing: 100%|██████████| 100/100 [00:14<00:00,  7.10it/s]


Test loss: 2353284.2125, size: 100


Testing: 100%|██████████| 100/100 [00:14<00:00,  7.08it/s]


...

Test loss: 77.54559627532959, size: 100


Testing: 100%|██████████| 100/100 [00:13<00:00,  7.36it/s]


Test loss: 78.5699792098999, size: 100


Testing: 100%|██████████| 100/100 [00:13<00:00,  7.33it/s]

Test loss: 78.08197334289551, size: 100


In [13]:
#Made larger network
class LargerYearNetwork(nn.Module):
    def __init__(self,input_size):
        super().__init__()

        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.BatchNorm1d(input_size),
            nn.Linear(input_size,256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128,128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Linear(32,16),
            nn.ReLU(),
            nn.BatchNorm1d(16),
            nn.Linear(16,1),
            
        )

    def forward(self,x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)

        return x

In [14]:
batch_size=256
modelLarge=LargerYearNetwork(df.shape[1]-1)
train_dataloaderLarge = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,drop_last=True)
test_dataloaderLarge = DataLoader(test_dataset, batch_size=batch_size, shuffle=True,drop_last=True)

optimizer=optim.Adam(modelLarge.parameters(),lr=lr)

In [15]:
#larger Network with 256 batch size
train_network(model=modelLarge,train_dataloader=train_dataloaderLarge,test_dataloader=test_dataloaderLarge,optimizer=optimizer)

Testing: 100%|██████████| 201/201 [00:14<00:00, 14.21it/s]


Test loss: 2351738.7537313434, size: 201


Testing: 100%|██████████| 201/201 [00:13<00:00, 14.38it/s]


Test loss: 413524.0729166667, size: 201


Testing: 100%|██████████| 201/201 [00:14<00:00, 13.73it/s]


...

Test loss: 77.73921499679338, size: 201


Testing: 100%|██████████| 201/201 [00:14<00:00, 13.96it/s]


Test loss: 78.04404198945458, size: 201


Testing: 100%|██████████| 201/201 [00:14<00:00, 13.66it/s]

Test loss: 79.08563042635942, size: 201


In [34]:
def l1_test_loop(model,dataloader):
    model.eval()
    size = len(dataloader)
    test_loss=0
    with torch.no_grad():
        for inputs,labels in tqdm(dataloader, desc="Testing: "):
            pred=model(inputs)

            loss_item=F.l1_loss(torch.ravel(pred),torch.ravel(labels)).item()
            test_loss+=loss_item
            
            
    test_loss=test_loss / size
    print(f'Test loss: {test_loss}')

In [35]:
#How many years is the 256 batch size model off by on average
l1_test_loop(model=model256,dataloader=test_dataloader256)

Testing: 100%|██████████| 201/201 [00:13<00:00, 15.11it/s]

Test loss: 6.052619865284631


In [108]:
#How often does the model get the decade of the song correct
def correct_decades(prediction,actual):
        #we know the songs range from 1922 to 2011
        prediction=[2011 if x > 2011 else 1922 if x < 1922 else x for x in prediction]
        prediction=[x%100//10 for x in prediction]
        actual=[x%100//10 for x in actual]

        return sum(a==b for a,b in zip(prediction,actual))
        
decade_dataloader = DataLoader(test_dataset, batch_size=256,drop_last=True)
# x,y=next(iter(decade_dataloader))
# y_hat=model256(x)
# print(y_hat.flatten().tolist())
# print(y.flatten())


h=[1969.7,85,2099.7,2000.4,1989]
i=[1965,1924.1,2011.2,2003.6,1999]
print(correct_decades(h,i))

model256.eval()
total=len(decade_dataloader.dataset)
correct=0

with torch.no_grad():
    for inputs,labels in tqdm(decade_dataloader):
        pred_years=model256(inputs).flatten().tolist()

        actual_years=labels.flatten().tolist()
        correct += correct_decades(pred_years,actual_years)

print(f"Decades Accuracy: {correct/total*100}% ({correct}/{total})")

4


100%|██████████| 201/201 [00:13<00:00, 14.64it/s]

Decades Accuracy: 58.343986054619414% (30123/51630)


# Great!
The model gets within about 6 years of when the song was released on average, and as a bonus can correctly guess which decade the song is from more than half the time